### Getting Started

In [9]:
import keyring

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus

# Use paper trading, no real money used.
USE_PAPER = True

(api_key_type, secret_key_type) = ('api_key_paper', 'secret_key_paper') if USE_PAPER else ('api_key', 'secret_key')
api_key = keyring.get_password('alpaca', api_key_type)
secret_key = keyring.get_password('alpaca', secret_key_type)

if not api_key or not secret_key:
    raise Exception('Alpaca API key not found.')
trading_client = TradingClient(api_key, secret_key, paper=USE_PAPER)

# Get our account information.
account = trading_client.get_account()

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

# Check our current balance vs. our balance at the last market close
balance_change = float(account.equity) - float(account.last_equity)
print(f'Today\'s portfolio balance change: ${balance_change}')

$199938.85663 is available as buying power.
Today's portfolio balance change: $0.0


In [6]:
# Search for AAPL
aapl_asset = trading_client.get_asset('AAPL')

if aapl_asset.tradable:
    print('We can trade AAPL.')

We can trade AAPL.


In [6]:
# Quantity market order
market_order_data = MarketOrderRequest(
                    symbol='AAPL',
                    qty=0.023,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )

# Notional market order
market_order_data = MarketOrderRequest(
                    symbol='AAPL',
                    notional=20,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY
                    )

# Market order
market_order = trading_client.submit_order(
                order_data=market_order_data
               )

In [14]:
# params to filter orders by
request_params = GetOrdersRequest(
                    status=QueryOrderStatus.OPEN,
                    side=OrderSide.BUY
                 )

# orders that satisfy params
orders = trading_client.get_orders(filter=request_params)

orders

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
     'canceled_at': None,
     'client_order_id': '58d5e0ab-2b24-4d8e-93bf-35022228c57e',
     'created_at': datetime.datetime(2023, 8, 12, 23, 54, 36, 421320, tzinfo=datetime.timezone.utc),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('d38316e6-ff4f-404c-9f6f-2b5ddd06abe3'),
     'legs': None,
     'limit_price': None,
     'notional': '20',
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.MARKET: 'market'>,
     'qty': None,
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.ACCEPTED: 'accepted'>,
     'stop_price': None,
     'submitted_at': datetime.datetime(2023, 8, 12, 23, 54, 36, 420421, tzinfo=dat

In [15]:
cancel_statuses = trading_client.cancel_orders()

In [16]:
trading_client.get_all_positions()

[{   'asset_class': <AssetClass.US_EQUITY: 'us_equity'>,
     'asset_id': UUID('b0b6dd9d-8b9b-48a9-ba46-b9d54906e415'),
     'asset_marginable': True,
     'avg_entry_price': '178.47475',
     'avg_entry_swap_rate': None,
     'change_today': '0',
     'cost_basis': '32.839354',
     'current_price': '177.79',
     'exchange': <AssetExchange.NASDAQ: 'NASDAQ'>,
     'lastday_price': '177.79',
     'market_value': '32.71336',
     'qty': '0.184',
     'qty_available': '0.184',
     'side': <PositionSide.LONG: 'long'>,
     'swap_rate': None,
     'symbol': 'AAPL',
     'unrealized_intraday_pl': '0',
     'unrealized_intraday_plpc': '0',
     'unrealized_pl': '-0.125994',
     'unrealized_plpc': '-0.0038366771770236',
     'usd': None}]

### Market Data

In [21]:
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest, StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

stock_client = StockHistoricalDataClient(api_key, secret_key)

# multi symbol request - single symbol is similar
multisymbol_request_params = StockLatestQuoteRequest(symbol_or_symbols=['NVDA', 'AMD'])

latest_multisymbol_quotes = stock_client.get_stock_latest_quote(multisymbol_request_params)

latest_multisymbol_quotes['NVDA'].ask_price


492.67

In [22]:
request_params = StockBarsRequest(
                        symbol_or_symbols=['NVDA', 'AMD'],
                        timeframe=TimeFrame.Day,
                        start=datetime(2022, 7, 1),
                        end=datetime(2022, 9, 1)
                 )

bars = stock_client.get_stock_bars(request_params)

# convert to dataframe
bars.df

# access bars as list - important to note that you must access by symbol key
# even for a single symbol request - models are agnostic to number of symbols
bars['NVDA']

[{   'close': 145.23,
     'high': 150.6291,
     'low': 143.92,
     'open': 148.99,
     'symbol': 'NVDA',
     'timestamp': datetime.datetime(2022, 7, 1, 4, 0, tzinfo=datetime.timezone.utc),
     'trade_count': 624463.0,
     'volume': 57756955.0,
     'vwap': 146.209776},
 {   'close': 149.64,
     'high': 149.71,
     'low': 140.55,
     'open': 141.75,
     'symbol': 'NVDA',
     'timestamp': datetime.datetime(2022, 7, 5, 4, 0, tzinfo=datetime.timezone.utc),
     'trade_count': 619576.0,
     'volume': 65220761.0,
     'vwap': 146.280709},
 {   'close': 151.3,
     'high': 153.1899,
     'low': 147.89,
     'open': 150.1,
     'symbol': 'NVDA',
     'timestamp': datetime.datetime(2022, 7, 6, 4, 0, tzinfo=datetime.timezone.utc),
     'trade_count': 466923.0,
     'volume': 52906586.0,
     'vwap': 150.767714},
 {   'close': 158.58,
     'high': 159.45,
     'low': 153.89,
     'open': 154.555,
     'symbol': 'NVDA',
     'timestamp': datetime.datetime(2022, 7, 7, 4, 0, tzinfo=date

### Trading

In [4]:
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass

# search for crypto assets
search_params = GetAssetsRequest(asset_class=AssetClass.CRYPTO)

assets = trading_client.get_all_assets(search_params)

assets

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('9226ef75-ba41-4a4b-a1b4-96e1a1b818f0'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.052384095,
     'min_trade_increment': 1e-09,
     'name': 'Solana / US Dollar',
     'price_increment': 0.0025,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'SOL/USD',
     'tradable': False},
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('cd86e777-f976-4a41-9878-c54c80dab107'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 8.741258741,
     'min_trade_increment': 1e-09,
     'name': 'Algorand / USD Tether',
     'pr